<font size="4">This notebook will compute values for GPLLJ criteria adopted from [Doubler et al. (2015)][1].:</font>

[1]: https://doi.org/10.1175/JAMC-D-14-0311.1

In [1]:
import numpy as np
import xarray as xr
from datetime import datetime
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
from scipy.signal import argrelextrema

In [2]:
import cartopy
cartopy.config['pre_existing_data_dir']='/ihesp/shared/cartopy_features'
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER

#### Read LR wind data into memory

In [3]:
#-- LRMIP 6-hourly (instantaneous) wind data - regridded
#-- original path = '/ihesp/archive/LR/b.e13.B1950TRC5.ne30_g16.ihesp24_1950-2050.002/atm/hist/'

year = '1989'

path = '/ihesp/user/asblack/regridded/'
f_U_1 = 'cam_h3_LR_U_'+year+'052700Z_'+year+'061400Z.nc'
f_U_2 = 'cam_h3_LR_U_'+year+'061406Z_'+year+'070206Z.nc'
f_U_3 = 'cam_h3_LR_U_'+year+'070212Z_'+year+'072012Z.nc'
f_U_4 = 'cam_h3_LR_U_'+year+'072018Z_'+year+'080718Z.nc'
f_U_5 = 'cam_h3_LR_U_'+year+'080800Z_'+year+'082600Z.nc'
f_U_6 = 'cam_h3_LR_U_'+year+'082606Z_'+year+'091306Z.nc'

f_V_1 = 'cam_h3_LR_V_'+year+'052700Z_'+year+'061400Z.nc'
f_V_2 = 'cam_h3_LR_V_'+year+'061406Z_'+year+'070206Z.nc'
f_V_3 = 'cam_h3_LR_V_'+year+'070212Z_'+year+'072012Z.nc'
f_V_4 = 'cam_h3_LR_V_'+year+'072018Z_'+year+'080718Z.nc'
f_V_5 = 'cam_h3_LR_V_'+year+'080800Z_'+year+'082600Z.nc'
f_V_6 = 'cam_h3_LR_V_'+year+'082606Z_'+year+'091306Z.nc'

filenames_U = (path+f_U_1, path+f_U_2, path+f_U_3,
               path+f_U_4, path+f_U_5, path+f_U_6)
filenames_V = (path+f_V_1, path+f_V_2, path+f_V_3,
               path+f_V_4, path+f_V_5, path+f_V_6)

In [4]:
ds_U = xr.open_mfdataset(filenames_U,
                       engine='netcdf4',
                       parallel=True,
                       concat_dim='time',
                       data_vars='minimal',
                       coords='minimal',
                       compat='override').U.isel(lev=slice(1,13))

ds_V = xr.open_mfdataset(filenames_V,
                       engine='netcdf4',
                       parallel=True,
                       concat_dim='time',
                       data_vars='minimal',
                       coords='minimal',
                       compat='override').V.isel(lev=slice(1,13))

In [5]:
#-- Slice over JJA months
#-- N.B. must slice by time index position instead of label (.sel throws error)
ds_U = ds_U.isel(time=slice(20,388))
ds_V = ds_V.isel(time=slice(20,388))

In [6]:
#-- Slice over GoM region
global_offset = 360
ds_U = ds_U.sel(lat=slice(15,55), lon=slice(global_offset-120,global_offset-70))
ds_V = ds_V.sel(lat=slice(15,55), lon=slice(global_offset-120,global_offset-70))

In [7]:
start = datetime.now()

ds_U.load()
ds_V.load()

print('Done reading data into memory:', datetime.now() - start)

Done reading data into memory: 0:00:00.584979


In [8]:
ds_UV = xr.Dataset({
    'U': xr.DataArray(
                data   = ds_U.data,
                dims   = ['time', 'lev', 'lat', 'lon'],
                coords = {'time': ds_U.time.data,
                          'lev': ds_U.lev.data,
                          'lat': ds_U.lat.data,
                          'lon': ds_U.lon.data},
                attrs  = {
                    'description': 'Zonal wind',
                    'units'     : 'm/s'
                    }
                ),
    'V': xr.DataArray(
                data   = ds_V.data,
                dims   = ['time', 'lev', 'lat', 'lon'],
                coords = {'time': ds_V.time.data,
                          'lev': ds_V.lev.data,
                          'lat': ds_V.lat.data,
                          'lon': ds_V.lon.data},
                attrs  = {
                    'description': 'Meridional wind',
                    'units'     : 'm/s'
                    }
                )
            },
        attrs = {'description': 'LRMIP 6-hourly U,V components, regridded to lat/lon'}
    )

In [9]:
#-- Write to output file
path = '/ihesp/user/asblack/regridded/'
filename = 'cam_h3_LR_U_V_'+year+'060100Z_'+year+'083118Z.nc'
ds_UV.to_netcdf(path + filename, format="NETCDF4_CLASSIC")

### Criterion 1: maximum wind speed at or below 700 hPa is ≥ 12 m/s

In [10]:
#-- Compute wind speed on resampled wind data
wind_speed = np.sqrt( np.square(ds_U) + np.square(ds_V) )

#-- Compute wind maxima and level of wind maxima at each grid point below 700 hPa
max_val = wind_speed.isel(lev=slice(2, 12)).max(dim='lev')
max_level = wind_speed.isel(lev=slice(2, 12)).idxmax(dim='lev')

### Criterion 2: wind direction at level of maximum wind speed is southerly (113° and 247°)

In [11]:
#-- Compute wind direction at the level of wind maxima for each grid point
uu = ds_U.sel(lev=max_level)
vv = ds_V.sel(lev=max_level)
wind_dir = np.mod(180+180/np.pi*np.arctan2(uu,vv), 360)

criteria_2 = xr.zeros_like(wind_dir)
#-- Make any non-southerly wind direction value equal to zero
criteria_2 = wind_dir.where(wind_dir >= 113.0, 0).where(wind_dir <= 247, 0)
#-- Make southerly wind direction values equal to one
criteria_2 = criteria_2.where(criteria_2 == 0, 1)

### Criterion 3: wind speed decreased by ≥6 m/s from the level of maximum wind speed to the next minimum above or to 550 hPa (whichever was lower)

In [12]:
#-- Pad values of wind speed below the maximum as the maximum value (a hack to avoid uneven slicing to find first minimum above)
ws_padded = wind_speed.where(wind_speed.lev<=wind_speed.isel(lev=slice(2, 12)).idxmax(dim='lev'), max_val)

In [13]:
#-- Use scipy to locate all local minima in vertical column above the wind max, saving the FIRST minimum above the max
#-- If no minima exist above the wind max, the 550 level (idx=0) is saved
#-- N.B. levels are reversed for this data, hence the index of -1
start = datetime.now()

first_local_min_idx = xr.zeros_like(ws_padded.isel(lev=0))

for time_idx in np.arange(0, len(ws_padded.time)):
    for lat_idx in np.arange(0, len(ws_padded.lat)):
        for lon_idx in np.arange(0, len(ws_padded.lon)):
                               
            local_min_idxs = argrelextrema(ws_padded[time_idx,:,lat_idx,lon_idx].values, np.less)
        
            if (np.any(local_min_idxs) == True):
                first_local_min_idx[time_idx,lat_idx,lon_idx] = local_min_idxs[0][-1]

#-- Convert to integers
first_local_min_idx = first_local_min_idx.astype(int)

print('Done computing:', datetime.now() - start)

Done computing: 0:09:19.981004


In [14]:
#-- Find minimum wind speed values at the level of first minimum above maximum
min_above_max = ws_padded.isel(lev=first_local_min_idx)

In [15]:
#-- Compute the difference: this value should be >= 6 m/s in order to pass the jet criterion 3
diff_above = max_val - min_above_max

### Criterion 4: wind speed decreased by ≥6 m/s below the level of maximum wind speed

In [16]:
#-- Isolate wind speed below 700 hPa
ws_below700 = wind_speed.isel(lev=slice(2,12))

In [17]:
#-- Pad values of wind speed above the maximum as the maximum value (a hack to avoid uneven slicing to find minimum)
ws_below700_padded = ws_below700.where(ws_below700.lev>=ws_below700.idxmax(dim='lev'), max_val)

In [18]:
#-- Compute minimum below the maximum
min_below_max = ws_below700_padded.min(dim='lev')

In [19]:
#-- Compute the difference: this value should be >= 6 m/s in order to pass the jet criterion 4
diff_below = max_val - min_below_max

### Write output to netcdf file

In [20]:
ds_jet = xr.Dataset({
    'wind_max': xr.DataArray(
                data   = max_val.data,
                dims   = ['time', 'lat', 'lon'],
                coords = {'time': max_val.time.data,
                          'lat': max_val.lat.data,
                          'lon': max_val.lon.data},
                attrs  = {
                    'description': 'Maximum wind value at or below 700 hPa',
                    'units'     : 'm/s'
                    }
                ),
    'criteria_2_flag': xr.DataArray(
                data   = criteria_2.data,
                dims   = ['time', 'lat', 'lon'],
                coords = {'time': criteria_2.time.data,
                          'lat': criteria_2.lat.data,
                          'lon': criteria_2.lon.data},
                attrs  = {
                    'description': 'Value of 1 where wind is southerly (113-247 deg) at level of wind maximum',
                    'units'     : 'binary_flag'
                    }
                ),
    'diff_above': xr.DataArray(
                data   = diff_above.data,
                dims   = ['time', 'lat', 'lon'],
                coords = {'time': diff_above.time.data,
                          'lat': diff_above.lat.data,
                          'lon': diff_above.lon.data},
                attrs  = {
                    'description': 'wind speed difference between level of maximum wind speed and the next minimum above (or to 550 hPa, whichever was lower)',
                    'units'     : 'm/s'
                    }
                ),
    'diff_below': xr.DataArray(
                data   = diff_below.data,
                dims   = ['time', 'lat', 'lon'],
                coords = {'time': diff_below.time.data,
                          'lat': diff_below.lat.data,
                          'lon': diff_below.lon.data},
                attrs  = {
                    'description': 'wind speed difference between level of maximum wind speed and the minimum below',
                    'units'     : 'm/s'
                    }
                )
            },
        attrs = {'description': 'Computed from LRMIP 6-hourly u,v components'}
    )

In [21]:
#-- Write to output file
path = '/ihesp/user/asblack/GPLLJ/'
filename = 'cam_h3_LR_6-hourly_jet_info_'+year+'.nc'
ds_jet.to_netcdf(path + filename, format="NETCDF4_CLASSIC")